<a href="https://colab.research.google.com/github/rockatoo/durable_rules_task/blob/main/%EC%82%B0%EC%97%85%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EA%B0%9C%EB%A1%A0_5%EC%A3%BC%EC%B0%A8_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install durable_rules

In [2]:
from durable.lang import *

주문 클래스



In [3]:
class Product:
  def __init__(self, name, price):
    self.name = name
    self.price = price
  
  def get_name(self):
    return self.name
  
  def get_price(self):
    return self.price

class Products(Product):
  def __init__(self, product, ea):
    super().__init__(product.name, product.price)
    self.ea = ea
  
  def get_total_price(self):
    return self.price * self.ea

class ProductsList:
  def __init__(self, products_tuple):
    self.products_tuple = list(products_tuple) if isinstance(products_tuple, (tuple, list)) else list()
  
  def get_total_price(self):
    total_price = 0
    for products in self.products_tuple:
      total_price += products.get_total_price()
    
    return total_price

class Member:
  def __init__(self, name, mileage):
    self.name = name
    self.mileage = mileage
  
  def get_name(self):
    return self.name

  def get_mileage(self):
    return self.mileage

class Order:
  def __init__(self, products_list, payment_price, using_mileage, member):
    self.products_list = products_list
    self.payment_price = payment_price
    self.using_mileage = using_mileage
    self.member = member
  
  def get_products_price(self):
    return self.products_list.get_total_price()
  
  def get_using_mileage(self):
    return self.using_mileage
  
  def get_payment_price(self):
    return self.payment_price
  
  def get_member(self):
    return self.member

In [4]:
products_tuple = (
    Products(Product('웨하스', 3900), 4),
    Products(Product('단백질 보충제', 18900), 2),
    Products(Product('커피믹스 60개', 8000), 3)
)
products_list = ProductsList(products_tuple)
member = Member('강형욱', 6900)
order = Order(products_list, 76900, 500, member)

with ruleset('Order Validation'):
    @when_all(
        (m.products_price > 0) &
        (m.payment_price >= 0) &
        (m.using_mileage >= 0))
    def parameters(c):
      print('parameters have no problem')

    @when_all(m.calculated_price == m.products_price)
    def payment(c):
      print('payment is correct')
    
    @when_all(m.member_mileage >= m.using_mileage)
    def mileage(c):
      print('using mileage is correct')
    
    @when_all(+m.member_name)
    def member_name(c):
      print('member exists')

In [5]:
post('Order Validation', {
    'products_price': order.get_products_price(),
    'payment_price': order.get_payment_price(),
    'using_mileage': order.get_using_mileage()
    })
post('Order Validation', {
    'products_price': order.get_products_price(),
    'calculated_price': order.get_payment_price() + order.get_using_mileage()
    })
post('Order Validation', {
    'member_mileage': order.get_member().get_mileage(),
    'using_mileage': order.get_using_mileage()
    })
post('Order Validation', {'member_name': order.get_member().get_name()})

parameters have no problem
payment is correct
using mileage is correct
member exists


{'$s': 1, 'id': 'sid-0', 'sid': '0'}